# Installing Libs

In [1]:
!pip install datasets

In [2]:
!pip install openai

In [3]:
!pip install sentence_transformers

# Importing Libs

In [2]:
from huggingface_hub import login
from datasets import load_dataset, concatenate_datasets, Dataset
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import openai
import numpy as np
import json
import os
from sentence_transformers import SentenceTransformer

# Importing Data

In [3]:
embed_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
login(token="TOKEN_HERE")
dataset = load_dataset("Ktzoras/shipping_news_articles", split = 'train')
df = pd.DataFrame(dataset)
type(df.date.iloc[0])
del dataset


README.md:   0%|          | 0.00/541 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/237M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40013 [00:00<?, ? examples/s]

In [5]:
dataset = load_dataset('Ktzoras/shipping_literature_embeddings', split = 'train')
dataset = pd.DataFrame(dataset)

README.md:   0%|          | 0.00/398 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4147 [00:00<?, ? examples/s]

# Deepseek

In [ ]:
api_key = 'API_KEY_HERE'
client = openai.OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com",
)

## Deepseek Functions

In [7]:
def get_response_JSON(messages):
  response = client.chat.completions.create(
          messages = messages,
          temperature = 0,
          max_tokens = 100,
          top_p = 1,
          frequency_penalty = 0,
          presence_penalty = 0,
          model="deepseek-chat",
          response_format = {
              'type': 'json_object'
              }
      )
  return response

def get_response(messages):
  response = client.chat.completions.create(
          messages = messages,
          temperature = 0,
          max_tokens = 100,
          top_p = 1,
          frequency_penalty = 0,
          presence_penalty = 0,
          model="deepseek-chat",
      )
  return response

####
# Function to extract information - FE
def extract_fe(article, prompt_fe):
    messages = [{"role": "system", "content": prompt_fe},
                {"role": "user", "content": article}]
    completion_response = get_response_JSON(messages)
    return json.loads(completion_response.choices[0].message.content)
####
def get_top_chunks(embed_model, article, dataset):
    article_embedding = embed_model.encode(article, convert_to_numpy = True)
    chunk_embeddings = np.vstack(dataset["embedding"].to_list())
    similarities = cosine_similarity([article_embedding], chunk_embeddings)[0]
    top_k_indices = similarities.argsort()[-10:][::-1]
    # Return a list of dictionaries, each containing the chunk text
    # Create a list of dictionaries containing the chunk text
    # print(top_k_indices)
    # print(type(top_k_indices))
    # print(dataset.iloc[top_k_indices[0]])
    # print(dataset.iloc[top_k_indices[0]]['chunk_text'])
    # top_chunks = [{'chunk_text': dataset.iloc[i]["chunk_text"]} for i in top_k_indices]
    # return top_chunks, top_k_indices
    return top_k_indices

def extract_rag(article, prompt_with_context, dataset, embed_model):
    # 2. Find top-k similar chunks
    # top_chunks, top_k_indices = get_top_chunks(embed_model, article, dataset)
    top_k_indices = get_top_chunks(embed_model, article, dataset)

    # 3. Combine chunks into context
    # Accessing the 'chunk_text' key of the dictionary in top_chunks
    top_chunks = [{'chunk_text': dataset.iloc[i]["chunk_text"]} for i in top_k_indices]
    context = "\n\n".join([chunk['chunk_text'] for chunk in top_chunks])

    # 4. Format the full prompt
    full_prompt = prompt_with_context.format(article_text = article, context = context)

    # 5. Send to DeepSeek chat API
    messages = [
        {"role": "system", "content": full_prompt},
        {"role": "user", "content": article}
    ]

    # Pass 'messages' instead of 'get_response' to 'get_response' function
    completion_response = get_response_JSON(messages)
    return json.loads(completion_response.choices[0].message.content), top_k_indices
####
# Selection using DeepSeek
def pick_best_chunks(article, candidate_chunks, selection_prompt):
    numbered_chunks = "\n\n".join([f"Chunk {i+1}: {chunk}" for i, chunk in enumerate(candidate_chunks)])

    messages = [{"role": "system", "content": selection_prompt}]
    selection_response = get_response(messages)

    response_text = selection_response.choices[0].message.content.strip()
    # Subtract 1 from each index to align with Python's 0-based indexing
    selected_indices = [int(x.strip())-1 for x in response_text.split(",") if x.strip().isdigit()]
    return selected_indices

# Function to extract information - IDFE
def extract_idfe(article, prompt_with_context, dataset, embed_model):
    # 2. Find top-k similar chunks
    # top_chunks, top_k_indices = get_top_chunks(embed_model, article, dataset)
    top_k_indices = get_top_chunks(embed_model, article, dataset)

    top_chunks = [{'chunk_text': dataset.iloc[i]["chunk_text"]} for i in top_k_indices]
    # 3. Get Context
    # Pass the top_chunks list to pick_best_chunks
    selected_indices = pick_best_chunks(article, [chunk['chunk_text'] for chunk in top_chunks], selection_prompt) # Pass list of chunk texts
    # Accessing elements using selected_indices
    # print(selected_indices)
    # print(type(selected_indices))
    # print(dataset.iloc[selected_indices[0]])
    # print(dataset.iloc[selected_indices[0]]['chunk_text'])
    # print(type(top_chunks))
    # print(len(top_chunks))
    # print(top_chunks[0])
    selected_chunks = [dataset.iloc[i]['chunk_text'] for i in selected_indices] # Access chunk texts from dictionaries
    context = "\n\n".join(selected_chunks)

    # 4. Create prompt
    prompt = prompt_with_context.format(context=context, article_text=article) # Assuming prompt_with_context is a string with placeholders

    messages = [{"role": "system", "content": prompt},
                {"role": "user", "content": article}]

    completion_response = get_response_JSON(messages) # Pass messages to get_response

    return json.loads(completion_response.choices[0].message.content), top_k_indices, selected_indices
####
def extract_rb(article, summary_prompt):
    messages = [{"role": "system", "content": summary_prompt}, {"role": "user", "content": article}]
    response = get_response(messages)
    return response.choices[0].message.content.strip()

def llm_pick_chunks(article, top_chunks, key_point_chunks_prompt):
    numbered_chunks = "\n\n".join([f"{i + 1}. {chunk}" for i, chunk in enumerate(top_chunks)])
    # prompt = key_point_chunks_prompt(article, numbered_chunks)
    prompt = key_point_chunks_prompt.format(key_points=extract_rb(article, summary_prompt), numbered_chunks=numbered_chunks)
    messages = [{"role": "system", "content": prompt}, {"role": "user", "content": article}]
    response = get_response(messages)

    # Extract indices and return actual chunks
    selected_indices = [int(i.strip()) - 1 for i in response.choices[0].message.content.strip().split(",") if i.strip().isdigit()]
    return [top_chunks[i] for i in selected_indices if 0 <= i < len(top_chunks)], selected_indices

# Function to extract information - IDRBFE
def extract_idrbfe(article, prompt_with_context_s2, dataset, embed_model, key_point_chunks_prompt):
    # 1. Get key points
    key_points = extract_rb(article, summary_prompt)

    # 2. Find top-k similar chunks
    # Pass key_points instead of dataset to embed_model.encode()
    top_k_indices = get_top_chunks(embed_model, key_points, dataset) #dataset is already being passed

    top_chunks = [{'chunk_text': dataset.iloc[i]["chunk_text"]} for i in top_k_indices]
    # Step 3: Use LLM to pick the most relevant 5–10 chunks
    selected_chunks, selected_indices = llm_pick_chunks(article, [chunk['chunk_text'] for chunk in top_chunks], key_point_chunks_prompt) # Pass list of chunk texts to llm_pick_chunks
    context = "\n\n".join(selected_chunks)

    # Step 4: Create final extraction prompt with article + context
    prompt = prompt_with_context_s2.format(context=context, article_text=article)  # Assuming prompt_with_context_s2 is a string with placeholders

    messages = [{"role": "system", "content": prompt}, {"role": "user", "content": article}]
    completion_response = get_response_JSON(messages)  # Pass messages to get_response

    # Step 5: Return JSON output
    return json.loads(completion_response.choices[0].message.content.strip()), top_k_indices, selected_indices

## Prompts

In [8]:
prompt_fe = '''
You are a maritime data analyst working for an international shipping research organization.
You are analyzing maritime news articles.

First, extract the following five attributes from the article:

1. **Scale** – The extent of the impact (Regional, National, Global).
2. **Type of Vessel** – The class or kind of vessel mentioned (Dry, Other).
3. **Vessel Size** – The vessel size category if specified (Capesize, Panamax, Supramax, Handysize, Other, All).
4. **Sea Route** – The affected sea route (C5, C3, All, Other).
5. **Duration of Positive Impact** – The length of time the positive effect is expected to last (Short, Medium, Long).
6. **Impact** – The direction of potential impact in hire rates (Negative, None, Positive).
7. **Hire rate impact** – Size/magnitude of that movement (None, Minor, Moderate, Significant, Major).

C5 is the route from Australia to China.
If any of the attributes cannot be determined from the article, say **None**.

News Article:

"{article_text}"

EXAMPLE JSON OUTPUT:
{"Scale": "National","Type of Vessel": "Other","Vessel Size": "All","Sea Route": "All","Duration of Positive Impact": "Long","Impact": "Negative","Hire Rate Impact": "Major"}
'''

prompt_rag = '''
You are a maritime data analyst working for an international shipping research organization.
You are analyzing maritime news articles, assisted by historical context extracted from shipping documents.

First, review the following **Context**:

{context}

From each article, extract the following attributes:

1. **Scale** – The extent of the impact (Regional, National, Global).
2. **Type of Vessel** – The class or kind of vessel mentioned (Dry, Other).
3. **Vessel Size** – The vessel size category if specified (Capesize, Panamax, Supramax, Handysize, Other, All).
4. **Sea Route** – The affected sea route (C5, C3, All, Other).
5. **Duration of Positive Impact** – The length of time the positive effect is expected to last (Short, Medium, Long).
6. **Impact** – The direction of potential impact in hire rates (Negative, None, Positive).
7. **Hire rate impact** – Size/magnitude of that movement (None, Minor, Moderate, Significant, Major).

C5 is the route from Australia to China.
If any of the attributes cannot be determined from the article, say **None**.

News Article:

"{article_text}"

EXAMPLE JSON OUTPUT:
{{"Scale": "National", "Type of Vessel": "Other", "Vessel Size": "All", "Sea Route": "All", "Duration of Positive Impact": "Long", "Impact": "Negative", "Hire Rate Impact": "Major"}}
'''


prompt_idfe = '''
You are a maritime data analyst working for an international shipping research organization.
You are analyzing maritime news articles, assisted by historical context extracted from shipping documents.

First, review the following **Context**:

{context}

From each article, infer and extract the following seven attributes:

1. **Scale** – The extent of the impact (Regional, National, Global).
2. **Type of Vessel** – The class or kind of vessel mentioned (Dry, Other).
3. **Vessel Size** – The vessel size category if specified (Capesize, Panamax, Supramax, Handysize, Other, All).
4. **Sea Route** – The affected sea route (C5, C3, All, Other).
5. **Duration of Positive Impact** – The length of time the positive effect is expected to last (Short, Medium, Long).
6. **Impact** – The direction of potential impact in hire rates (Negative, None, Positive).
7. **Hire Rate Impact** – The degree of potential impact in hire rates (None, Minor, Moderate, Significant, Major).

**Instructions:**
- If the article or context explicitly mentions a vessel size, extract it.
- If vessel size is not clearly stated, write **None**.
- If the article or context mentions positive operational improvements, increased demand, or optimism, **reasonably infer** a Minor or Moderate hire rate improvement.
- If no clear signals are found even after reviewing the context, write **None**.
- Be analytical — combine information from both the article and the context.

*C5 is the dry bulk sea route from Australia to China.*

News Article:

"{article_text}"

EXAMPLE JSON OUTPUT:
{{"Scale": "National", "Type of Vessel": "Dry", "Vessel Size": "Capesize", "Sea Route": "C5", "Duration of Positive Impact": "Short", "Impact": "Positive", "Hire Rate Impact": "Moderate"}}
'''

prompt_idrbfe = '''
You are a maritime data analyst at a global shipping research organization.

Review the following **Context** providing background on market conditions, vessel preferences, routes, and economic trends:

{context}

Based on the context and the article, infer and extract these seven attributes:

1. **Scale** (Regional, National, Global)
2. **Type of Vessel** (Dry, Other)
3. **Vessel Size** (Capesize, Panamax, Supramax, Handysize, Other, All)
4. **Sea Route** (C5, C3, All, Other)
5. **Duration of Positive Impact** (Short, Medium, Long)
6. **Impact** (Negative, None, Positive)
7. **Hire Rate Impact** (None, Minor, Moderate, Significant, Major)

**Instructions:**
- Use context and article facts together.
- If vessel size is unknown, write **None**.
- Infer Minor or Moderate hire rate improvement if positive developments are indicated.
- If fundamentals suggest major shifts, adjust your inference.
- If no clear signals are found, write **None**.

*Notes:*
- *C5 = Australia to China dry bulk route.*
- *C3 = Brazil to China iron ore route.*

News Article:

"{article_text}"

EXAMPLE JSON OUTPUT:
{{"Scale": "National", "Type of Vessel": "Dry", "Vessel Size": "Capesize", "Sea Route": "C5", "Duration of Positive Impact": "Short", "Impact": "Positive", "Hire Rate Impact": "Moderate"}}
'''



selection_prompt = '''
    You are a maritime economics expert.

    Given the candidate chunks:

    {numbered_chunks}

    Select **5–10** chunks that best provide economic fundamentals to understand shipping news.

    Focus on:
    - Freight demand drivers (iron ore, coal, grain)
    - Vessel types/sizes
    - Key routes (C5, C3) and seasonality
    - Hire rate patterns
    - Port congestion, trade disruptions
    - Long-term supply-demand shifts

    Guidelines:
    - Cover **different topics** if possible.
    - Prefer chunks with **explicit data or trends**.
    - Pick fewer if not enough are relevant.

    Reply format JSON:
    - **Only** list chunk numbers, comma-separated (e.g., `1, 5, 9`).
    - **No extra text.**
    - **No empty list, always at least 5 chunk numbers

    Prioritize relevance, facts, and diversity.
    '''

summary_prompt = """
You are a maritime economics analyst assisting a document retrieval system.

Given the following news article:

\"\"\"{article}\"\"\"

Your task:
- Summarize the article in **3 to 5 concise bullet points**.
- Focus on the **core facts and economic signals** that would help match this article with **similar maritime content**.
- The summary should **highlight key topics**, such as:
    - Trade routes
    - Vessel types/sizes
    - Market impacts
    - Freight rates
    - Regulatory or operational changes
    - Port activity or disruptions

Guidelines:
- Use **clear, information-dense language**.
- Avoid vague generalities.
- Structure your output as a **clean bullet list** — no intro or conclusion.

Your summary will be embedded and used to **improve cosine similarity search**, so **prioritize phrases that capture the essence of the article**.

Output format:
- Just bullet points (e.g. `- Maersk opens new Shanghai–LA route reducing transit by 5 days`)
"""

key_point_chunks_prompt = """
You are a maritime domain expert assisting an information extraction system.

Your goal is to select **5 to 10 chunks** from the list below that are most **relevant** to the article key points and useful for extracting the following attributes:
- Scale
- Type of Vessel
- Vessel Size
- Sea Route
- Duration of Positive Impact
- Impact
- Hire Rate Impact

Given this article key points:
\"\"\"{key_points}\"\"\"

And these candidate chunks:
{numbered_chunks}

Guidelines:
- Prefer chunks that provide **explicit facts**, **specific data**, or **clear descriptions**.
- Prioritize variety across topics (e.g., don't choose 5 route-related chunks only).
- Discard vague or repetitive chunks unless they uniquely support one of the attributes.

Output instructions:
- Reply **only** with the selected chunk numbers in a comma-separated list (e.g., `2, 5, 7, 9, 12`).
- Do **not** include explanations or extra text.
"""

## Running - Tests

In [9]:
extract_fe(df.content.iloc[10], prompt_fe)

{'Scale': 'Global',
 'Type of Vessel': 'Other',
 'Vessel Size': 'None',
 'Sea Route': 'None',
 'Duration of Positive Impact': 'Long',
 'Impact': 'Positive',
 'Hire Rate Impact': 'Moderate'}

In [12]:
# extract_rag(df.content.iloc[10], prompt_rag, dataset, embed_model)

In [13]:
# extract_idfe(df.content.iloc[10], prompt_idfe, dataset, embed_model)

In [14]:
# extract_idrbfe(df.content.iloc[10], prompt_idrbfe, dataset, embed_model, key_point_chunks_prompt)

# RUN for 5000

In [10]:
subsample = df[4000:5000]

In [11]:
subsample.shape

(1000, 7)

In [14]:
results = pd.DataFrame(columns = ["fe", "rag", "idfe", "idrbfe"])

In [16]:
for a, row in subsample[518:].iterrows():
  fe = extract_fe(row.content, prompt_fe)
  rag = extract_rag(row.content, prompt_rag, dataset, embed_model)
  idfe = extract_idfe(row.content, prompt_idfe, dataset, embed_model)
  idrbfe = extract_idrbfe(row.content, prompt_idrbfe, dataset, embed_model, key_point_chunks_prompt)
  results = pd.concat([results, pd.DataFrame({"fe": [fe], "rag": [rag], "idfe": [idfe], "idrbfe": [idrbfe]})], ignore_index=True)
  #results = pd.concat([results, pd.DataFrame({"rag": [rag], "idfe": [idfe]})], ignore_index=True)
  print(a)

4518
4519
4520
4521
4522
4523
4524
4525
4526
4527
4528
4529
4530
4531
4532
4533
4534
4535
4536
4537
4538
4539
4540
4541
4542
4543
4544
4545
4546
4547
4548
4549
4550
4551
4552
4553
4554
4555
4556
4557
4558
4559
4560
4561
4562
4563
4564
4565
4566
4567
4568
4569
4570
4571
4572
4573
4574
4575
4576
4577
4578
4579
4580
4581
4582
4583
4584
4585
4586
4587
4588
4589
4590
4591
4592
4593
4594
4595
4596
4597
4598
4599
4600
4601
4602
4603
4604
4605
4606
4607
4608
4609
4610
4611
4612
4613
4614
4615
4616
4617
4618
4619
4620
4621
4622
4623
4624
4625
4626
4627
4628
4629
4630
4631
4632
4633
4634
4635
4636
4637
4638
4639
4640
4641
4642
4643
4644
4645
4646
4647
4648
4649
4650
4651
4652
4653
4654
4655
4656
4657
4658
4659
4660
4661
4662
4663
4664
4665
4666
4667
4668
4669
4670
4671
4672
4673
4674
4675
4676
4677
4678
4679
4680
4681
4682
4683
4684
4685
4686
4687
4688
4689
4690
4691
4692
4693
4694
4695
4696
4697
4698
4699
4700
4701
4702
4703
4704
4705
4706
4707
4708
4709
4710
4711
4712
4713
4714
4715
4716
4717


In [94]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [26]:
print(dataset[])

SyntaxError: invalid syntax. Perhaps you forgot a comma? (1079668354.py, line 1)

In [17]:
results_backup = results.copy()

In [18]:
results_backup.shape

(1000, 4)

In [46]:
results_backup4 = results_backup2.copy()
results_backup3 = results.copy()

In [101]:
results_backup3.shape

(3050, 4)

In [100]:
results_backup4.shape

(1001, 4)

In [62]:
results_backup2.shape

(4000, 4)

In [32]:
results.shape

(2000, 4)

In [19]:
results.duplicated()

TypeError: unhashable type: 'dict'

In [59]:
results_backup2 = results_backup2[:-2]

In [57]:
results_backup2.head()

,fe,rag,idfe,idrbfe
0,"{'Scale': 'Global', 'Type of Vessel': 'Other',...","({'Scale': 'Global', 'Type of Vessel': 'All', ...","({'Scale': 'Global', 'Type of Vessel': 'Other'...","({'Scale': 'Global', 'Type of Vessel': 'Other'..."
1,"{'Scale': 'National', 'Type of Vessel': 'Other...","({'Scale': 'National', 'Type of Vessel': 'Othe...","({'Scale': 'National', 'Type of Vessel': 'Othe...","({'Scale': 'National', 'Type of Vessel': 'Othe..."
2,"{'Scale': 'Global', 'Type of Vessel': 'Other',...","({'Scale': 'Global', 'Type of Vessel': 'Other'...","({'Scale': 'Global', 'Type of Vessel': 'Other'...","({'Scale': 'Global', 'Type of Vessel': 'Other'..."
3,"{'Scale': 'None', 'Type of Vessel': 'Dry', 'Ve...","({'Scale': 'Global', 'Type of Vessel': 'Dry', ...","({'Scale': 'Global', 'Type of Vessel': 'Dry', ...","({'Scale': 'Global', 'Type of Vessel': 'Dry', ..."
4,"{'Scale': 'None', 'Type of Vessel': 'None', 'V...","({'Scale': 'Regional', 'Type of Vessel': 'Othe...","({'Scale': 'Regional', 'Type of Vessel': 'Othe...","({'Scale': 'Global', 'Type of Vessel': 'Other'..."


In [20]:
for result in results.iterrows():

    fe = result[1].fe
    rag = result[1].rag
    idfe = result[1].idfe
    idrbfe = result[1].idrbfe
    results_backup = pd.concat([results_backup, pd.DataFrame({"fe": [fe], "rag": [rag], "idfe": [idfe], "idrbfe": [idrbfe]})], ignore_index=True)

In [21]:
results.idfe.iloc[1010]

IndexError: single positional indexer is out-of-bounds

In [22]:
results.idfe.iloc[3010]

IndexError: single positional indexer is out-of-bounds

In [ ]:
# results.drop(1161, inplace = True)

In [70]:
df1.head(1)

NameError: name 'df1' is not defined

In [23]:
df1 = pd.DataFrame([])
df2 = pd.DataFrame([])
df3 = pd.DataFrame([])
df4 = pd.DataFrame([])
for row in results.iterrows():
    print(row[1].fe)
    df1 = pd.concat([df1, pd.DataFrame([row[1].fe])])
    df2 = pd.concat([df2, pd.DataFrame([row[1].rag[0]])])
    df3 = pd.concat([df3, pd.DataFrame([row[1].idfe[0]])])
    df4 = pd.concat([df4, pd.DataFrame([row[1].idrbfe[0]])])


df1.columns = [str(col) + '_fe' for col in df1.columns]
df2.columns = [str(col) + '_rag' for col in df2.columns]
df3.columns = [str(col) + '_idfe' for col in df3.columns]
# df4.drop(columns = [0,1,2], inplace = True)
df4.columns = [str(col) + '_idrbfe' for col in df4.columns]


{'Scale': 'National', 'Type of Vessel': 'Other', 'Vessel Size': 'None', 'Sea Route': 'All', 'Duration of Positive Impact': 'Medium', 'Impact': 'Positive', 'Hire Rate Impact': 'Moderate'}
{'Scale': 'Global', 'Type of Vessel': 'Other', 'Vessel Size': 'None', 'Sea Route': 'None', 'Duration of Positive Impact': 'Long', 'Impact': 'Positive', 'Hire Rate Impact': 'Significant'}
{'Scale': 'Global', 'Type of Vessel': 'Dry', 'Vessel Size': 'All', 'Sea Route': 'All', 'Duration of Positive Impact': 'Short', 'Impact': 'Positive', 'Hire Rate Impact': 'Moderate'}
{'Scale': 'Global', 'Type of Vessel': 'Other', 'Vessel Size': 'All', 'Sea Route': 'All', 'Duration of Positive Impact': 'Short', 'Impact': 'Negative', 'Hire Rate Impact': 'Moderate'}
{'Scale': 'Global', 'Type of Vessel': 'Other', 'Vessel Size': 'All', 'Sea Route': 'All', 'Duration of Positive Impact': 'None', 'Impact': 'None', 'Hire Rate Impact': 'None'}
{'Scale': 'Global', 'Type of Vessel': 'Other', 'Vessel Size': 'All', 'Sea Route': 'All',

In [24]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df = pd.DataFrame([])
df = pd.concat([df1, df2, df3, df4], axis = 1)
# df.drop(2000, inplace = True)

In [26]:
# df2 = df2.reset_index(drop=True)
# df3 = df3.reset_index(drop=True)
# df = pd.concat([df2, df3], axis = 1)

In [36]:
df.shape

(1000, 29)

In [25]:
df[df.duplicated()]

,Scale_fe,Type of Vessel_fe,Vessel Size_fe,Sea Route_fe,Duration of Positive Impact_fe,Impact_fe,Hire Rate Impact_fe,Scale_rag,Type of Vessel_rag,Vessel Size_rag,...,Duration of Positive Impact_idfe,Impact_idfe,Hire Rate Impact_idfe,Scale_idrbfe,Type of Vessel_idrbfe,Vessel Size_idrbfe,Sea Route_idrbfe,Duration of Positive Impact_idrbfe,Impact_idrbfe,Hire Rate Impact_idrbfe
20,None,None,None,None,None,None,None,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
41,Global,Other,None,Other,Long,Positive,Moderate,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
42,Global,Other,None,None,Long,Positive,Moderate,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
47,Global,Other,None,None,Long,Positive,Moderate,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
51,Global,Other,None,None,Long,Positive,Moderate,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948,Global,Other,All,Other,Long,Positive,Moderate,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
961,Global,Other,None,None,Long,Positive,Moderate,Global,Other,All,...,Long,Positive,Moderate,Global,Other,None,All,Long,Positive,Moderate
970,Global,Other,All,Other,Medium,Positive,Moderate,Global,Other,All,...,Short,Positive,Moderate,Global,Other,None,Other,Short,Positive,Moderate
971,None,None,None,None,None,None,None,Global,Other,All,...,Long,Positive,Moderate,Global,Other,All,All,Long,Positive,Moderate


In [22]:
subsample.shape

(1000, 7)

In [41]:
results.shape

(1000, 4)

In [49]:
df.shape

(1000, 37)

In [29]:
df_articles[2000:3000].shape

(1000, 7)

In [47]:
df = pd.concat([df_articles[2000:3000].reset_index(), df], axis = 1)

In [50]:
df.head(2)

,index,link,date,title,content,vectors_L6_v2,vectors_distil,classes_distil,Scale_fe,Type of Vessel_fe,...,Duration of Positive Impact_idfe,Impact_idfe,Hire Rate Impact_idfe,Scale_idrbfe,Type of Vessel_idrbfe,Vessel Size_idrbfe,Sea Route_idrbfe,Duration of Positive Impact_idrbfe,Impact_idrbfe,Hire Rate Impact_idrbfe
0,2000,https://www.hellenicshippingnews.com/47-govern...,2025-01-10,47 Governments and global industry jointly pro...,To achieve net zero greenhouse gas (GHG) emiss...,"[-0.01074416097253561, 0.08349835127592087, -0...","[[101, 2000, 6162, 5658, 5717, 16635, 3806, 10...",POSITIVE,Global,Other,...,Long,Positive,Moderate,Global,Other,All,All,Long,Positive,Moderate
1,2001,https://www.hellenicshippingnews.com/ocean-con...,2025-01-10,Ocean container shipping rates may fall after ...,Strikes at ports on the US East Coast and Gulf...,"[0.025385890156030655, -0.031143244355916977, ...","[[101, 9326, 2012, 8831, 2006, 1996, 2149, 226...",NEGATIVE,National,Other,...,Short,Negative,Minor,Global,Other,None,Other,Short,Positive,Moderate


In [96]:
dataset = Dataset.from_dict(subsample)

In [ ]:
login(token="TOKEN_HERE")
dataset.push_to_hub('shipping_llm_results_2k_sample')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ktzoras/shipping_llm_results_2k_sample/commit/57e810cbe7467a63f53c5fd66a7a8dcd5da1ab1b', commit_message='Upload dataset', commit_description='', oid='57e810cbe7467a63f53c5fd66a7a8dcd5da1ab1b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ktzoras/shipping_llm_results_2k_sample', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ktzoras/shipping_llm_results_2k_sample'), pr_revision=None, pr_num=None)

: 

In [28]:
sub = load_dataset('Ktzoras/shipping_llm_results_4k_sample', split = 'train')
sub = pd.DataFrame(sub)

In [29]:
sub.shape

(4000, 37)

In [30]:
df_copy = df.copy()

In [31]:
df_copy.shape

(1000, 29)

In [32]:
df = pd.concat([sub, df], axis = 0)

In [34]:
df.drop(columns = ['__index_level_0__'], inplace = True)

In [35]:
df.head(1)

,link,date,title,content,vectors_L6_v2,vectors_distil,classes_distil,Scale_fe,Type of Vessel_fe,Vessel Size_fe,...,Impact_idfe,Hire Rate Impact_idfe,Scale_idrbfe,Type of Vessel_idrbfe,Vessel Size_idrbfe,Sea Route_idrbfe,Duration of Positive Impact_idrbfe,Impact_idrbfe,Hire Rate Impact_idrbfe,Hire rate impact_rag
0,https://www.hellenicshippingnews.com/global-ln...,2025-04-18,Global LNG freight falls sharply as Bintulu LN...,Global LNG freight tumbled sharply April 16 as...,"[0.019155355170369148, 0.013727688230574131, 0...","[[101, 3795, 1048, 3070, 8441, 18303, 9249, 22...",NEGATIVE,Global,Other,None,...,Negative,Significant,Global,Other,None,Other,None,Negative,Significant,None


In [36]:
sub.head()

,link,date,title,content,vectors_L6_v2,vectors_distil,classes_distil,Scale_fe,Type of Vessel_fe,Vessel Size_fe,...,Hire Rate Impact_idfe,Scale_idrbfe,Type of Vessel_idrbfe,Vessel Size_idrbfe,Sea Route_idrbfe,Duration of Positive Impact_idrbfe,Impact_idrbfe,Hire Rate Impact_idrbfe,Hire rate impact_rag,__index_level_0__
0,https://www.hellenicshippingnews.com/global-ln...,2025-04-18,Global LNG freight falls sharply as Bintulu LN...,Global LNG freight tumbled sharply April 16 as...,"[0.019155355170369148, 0.013727688230574131, 0...","[[101, 3795, 1048, 3070, 8441, 18303, 9249, 22...",NEGATIVE,Global,Other,None,...,Significant,Global,Other,None,Other,None,Negative,Significant,None,0
1,https://www.hellenicshippingnews.com/imos-net-...,2025-04-18,IMO’s net-zero strategy for global shipping st...,The IMO has adopted a framework that puts a pr...,"[0.00376713159494102, 0.07574530690908432, 0.0...","[[101, 1996, 10047, 2080, 2038, 4233, 1037, 77...",NEGATIVE,Global,Other,All,...,Moderate,Global,Other,All,All,None,None,None,None,1
2,https://www.hellenicshippingnews.com/landmark-...,2025-04-18,Landmark IMO deal set to shake up shipping,A decisive moment for global shipping regulati...,"[-0.03010372817516327, 0.04250948876142502, 0....","[[101, 1037, 13079, 2617, 2005, 3795, 7829, 78...",POSITIVE,Global,Other,All,...,Moderate,Global,Other,All,All,Long,Positive,Moderate,None,2
3,https://www.hellenicshippingnews.com/russian-a...,2025-04-18,Russian Arctic oil exports to China jump helpe...,Russia’s Arctic oil exports to China are set t...,"[-0.059621743857860565, -0.03635021299123764, ...","[[101, 3607, 1521, 1055, 10162, 3514, 14338, 2...",NEGATIVE,Global,Other,Other,...,Moderate,Global,Other,Other,Other,None,Positive,Moderate,None,3
4,https://www.hellenicshippingnews.com/baltic-in...,2025-04-18,Baltic Index Rises On Strong Rates Across Vessels,The Baltic Exchange’s dry bulk sea freight ind...,"[-0.01619306020438671, 0.004889817908406258, -...","[[101, 1996, 11275, 3863, 1521, 1055, 4318, 96...",NEGATIVE,Global,Dry,All,...,Moderate,Global,Dry,All,All,Short,Positive,Minor,None,4


In [37]:
for col in df.columns:
    sub[col] = df[col]

ValueError: cannot reindex on an axis with duplicate labels

In [38]:
sub.iloc[6]

link                                  https://www.hellenicshippingnews.com/imo-outco...
date                                                                2025-04-18 00:00:00
title                                 IMO: Outcomes of the Marine Environment Protec...
content                               The Marine Environment Protection Committee (M...
vectors_L6_v2                         [-0.03799080103635788, 0.03674209117889404, 0....
vectors_distil                        [[101, 1996, 3884, 4044, 3860, 2837, 1006, 203...
classes_distil                                                                 NEGATIVE
Scale_fe                                                                         Global
Type of Vessel_fe                                                                 Other
Vessel Size_fe                                                                      All
Sea Route_fe                                                                        All
Duration of Positive Impact_fe  

In [ ]:
dataset = Dataset.from_pandas(df)
login(token="TOKEN_HERE")
dataset.push_to_hub('shipping_llm_results_5k_sample')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ktzoras/shipping_llm_results_5k_sample/commit/f1ab5d2494bc2e71337763daf1b558e217ef3350', commit_message='Upload dataset', commit_description='', oid='f1ab5d2494bc2e71337763daf1b558e217ef3350', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ktzoras/shipping_llm_results_5k_sample', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ktzoras/shipping_llm_results_5k_sample'), pr_revision=None, pr_num=None)

In [40]:
type(sub)

pandas.core.frame.DataFrame

In [128]:
chunk_n = pd.DataFrame([])
for row in results.iterrows():
    # print(row[1].rag[1])
    chunk_n = pd.concat([chunk_n, pd.DataFrame(row[1].rag[1]).T], axis = 0)

In [129]:
chunk_n

,0,1,2,3,4,5,6,7,8,9
0,86,482,760,260,199,101,85,200,2148,3572
0,1564,1413,3567,3987,1560,1414,1161,1265,3582,1337
0,1564,894,1560,1160,3567,1313,1863,1271,3574,1578
0,379,4091,376,3793,437,4092,375,524,380,157
0,746,1883,1869,2137,86,785,101,980,203,1884
...,...,...,...,...,...,...,...,...,...,...
0,55,59,823,962,15,3983,812,1059,821,1989
0,1564,894,1160,1846,1560,901,3567,3584,2870,2853
0,3097,1564,3442,1559,3083,1160,2908,3574,3419,3582
0,86,1883,746,210,1878,1869,135,1884,575,101


In [ ]:
dataset = Dataset.from_pandas(chunk_n)
login(token="TOKEN_HERE")
dataset.push_to_hub('shipping_rag_results_2k_sample')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Ktzoras/shipping_rag_results_2k_sample/commit/fc776d6fc2f534e3276962f1f5df32c7cc987a76', commit_message='Upload dataset', commit_description='', oid='fc776d6fc2f534e3276962f1f5df32c7cc987a76', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Ktzoras/shipping_rag_results_2k_sample', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Ktzoras/shipping_rag_results_2k_sample'), pr_revision=None, pr_num=None)

In [38]:
dataset = load_dataset('Ktzoras/shipping_llm_results_3k_sample', split = 'train')
dataset = pd.DataFrame(dataset)

In [ ]:
dataset = pd.concat([dataset, df], axis = 1)